## Pacotes

In [1]:
import numpy as np

## Kalman Fusion

In [2]:
class KalmanFusion:
    """
    Fusão escalar (Kalman) para RADNET:
      - Medidas: score do modelo (via probs) com N heurísticas normalizadas [0,1]
      - R por-fonte (modelo e cada heurística)
      - R adaptativo opcional por resíduo (EWMA)
      - Saídas auxiliares: p_forward, contention window (CW), rate

    Interpretação:
      - 0=otimo
      - 1=péssimo
    """
    def __init__(
        self,
        Q=0.01,                  # Variância do processo (quão rápido o "verdadeiro" estado pode mudar)
        R_model=0.10,            # Variância da medida do modelo (quanto desconfiamos do GNN)
        R_feats=None,            # Dict de variâncias por métrica heurística: {"etx":0.08,"delay":0.05,...}
        weights=(0.0, 0.5, 1.0), # Pesos p/ converter classes -> score: baixa, média, alta
        adapt_R=False,           # Se True, ajusta R online via resíduo (fonte mais ruidosa pesa menos)
        resid_beta=0.15,         # Suavização EWMA para o R adaptativo (0..1) – maior = adapta mais rápido
        p_bounds=(0.05, 0.95),   # Limites (min,max) para probabilidade de rebroadcast (evita extremos)
        cw_bounds=(16, 1024),    # Limites (min,max) da janela de contenção (CW) para backoff/jitter
        rate_max=1.0             # Taxa normalizada máxima (shaper); rate = rate_max * (1 - x)
    ):
        self.Q = float(Q)                   # Variância do processo: quão rápido o estado "real" pode mudar (↑Q = mais responsivo, menos suave)
        self.R_model = float(R_model)       # Variância da medida do MODELO (GNN): ↑R_model = menos peso para o modelo na fusão
        self.R_feats = dict(R_feats or {})  # Dicionário de variâncias das heurísticas por métrica (ex.: {"etx":0.08,...});
        self.weights = tuple(weights)       # Pesos para converter probs de 3 classes -> score contínuo (baixa, média, alta)
        self.adapt_R = bool(adapt_R)        # Liga/desliga R adaptativo (ajuste online das variâncias via resíduos)
        self.resid_beta = float(resid_beta) # Fator EWMA (0..1) para suavizar a atualização do R adaptativo (↑ = adapta mais rápido)

        self.x = None          # estado (score fundido) em [0,1]
        self.P = 1.0           # covariância inicial

        self.p_min, self.p_max = p_bounds       # Limites [min,max] p/ prob. de rebroadcast (clamp de p_forward).
        self.cw_min, self.cw_max = cw_bounds    # Limites [min,max] da contention window/backoff (score↑ ⇒ CW↑).
        self.rate_max = rate_max                # Teto da taxa normalizada; rate = rate_max * (1 - score).

        self.p_min, self.p_max = float(self.p_min), float(self.p_max)   # Garante tipos float para os limites de p_forward (clamp numérico consistente).
        self.cw_min, self.cw_max = int(self.cw_min), int(self.cw_max)   # Garante tipos int para CW (backoff em valores inteiros).

    def _probs_to_score(self, probs):
        """Converte {'baixa':p1,'media':p2,'alta':p3} -> score contínuo [0,1]."""
        w_baixa, w_media, w_alta = self.weights
        return (
            w_baixa * probs.get("baixa", 0.0) +
            w_media * probs.get("media", 0.0) +
            w_alta  * probs.get("alta", 0.0)
        )

    def _combine_measures(self, measures, variances):
        """
        Combina medidas escalares independentes por média ponderada
        com pesos = 1/variância. Retorna (z_comb, R_comb).
        """
        inv_vars = [1.0/v for v in variances]
        s = sum(inv_vars)
        z_comb = sum(m*w for m, w in zip(measures, inv_vars)) / s
        R_comb = 1.0 / s
        return z_comb, R_comb

    def update(self, probs_dict, feats_dict):
        """
        probs_dict: {'baixa':..,'media':..,'alta':..}
        feats_dict: {'etx':0..1,'delay':0..1,'busy':0..1,'queue':0..1, ...} normalizados
        """
        z_model = float(self._probs_to_score(probs_dict))
        measures = [z_model]
        variances = [self.R_model]

        for k in sorted(feats_dict.keys()):
            measures.append(float(feats_dict[k]))
            variances.append(float(self.R_feats.get(k, 0.10)))

        if self.x is None:
            self.x = float(np.mean(measures))
        x_pred = self.x
        P_pred = self.P + self.Q

        z_comb, R_comb = self._combine_measures(measures, variances)

        K = P_pred / (P_pred + R_comb)

        x_new = x_pred + K * (z_comb - x_pred)
        self.x = float(np.clip(x_new, 0.0, 1.0))

        self.P = (1.0 - K) * P_pred

        if self.adapt_R:
            e = [m - self.x for m in measures]
            eps = 1e-4
            self.R_model = max(
                (1 - self.resid_beta) * self.R_model + self.resid_beta * (e[0]**2),
                eps
            )
            for idx, k in enumerate(sorted(feats_dict.keys()), start=1):
                r_old = self.R_feats.get(k, 0.10)
                r_new = max((1 - self.resid_beta) * r_old + self.resid_beta * (e[idx]**2), eps)
                self.R_feats[k] = r_new

        return self.x

    def fused_cost(self):
        return -np.log(self.x + 1e-6)

    def controls(self, score=None):       
        s = float(self.x if score is None else score)
        s = float(np.clip(s, 0.0, 1.0))

        p_forward = np.clip(1.0 - s, self.p_min, self.p_max)
        cw = int(round(self.cw_min + (self.cw_max - self.cw_min) * s))
        rate = self.rate_max * (1.0 - s)

        return {"p_forward": float(p_forward), "cw": cw, "rate": float(rate)}

## Execução

In [3]:
# Probabilidades do modelo (3 classes)
probs = {'baixa': 0.163, 'media': 0.307, 'alta': 0.529}

# Heurísticas
etx = 3.0
Delay = 70       # ms
Busy_fraction = 0.55
Q_cur, Q_max = 3, 10

# Normalização
etx_cost   = np.clip((etx - 1) / (10 - 1), 0, 1)  # ETX_max=10
delay_cost = np.clip(Delay / 200, 0, 1)           
busy_cost  = np.clip(Busy_fraction, 0, 1)         
queue_cost = np.clip(Q_cur / Q_max, 0, 1)

feats = {
    "etx": etx_cost,
    "delay": delay_cost,
    "busy": busy_cost,
    "queue": queue_cost,
}

kf = KalmanFusion(
    Q=0.01,
    R_model=0.10,
    R_feats={"etx":0.08, "delay":0.05, "busy":0.04, "queue":0.06},
    adapt_R=True,  
    resid_beta=0.15,
    p_bounds=(0.1, 0.9),
    cw_bounds=(16, 1024),
    rate_max=1.0
)

x = kf.update(probs, feats)
cost = kf.fused_cost()
ctrl = kf.controls()

In [4]:
print("Score fundido:", x)
print("Custo final:", cost)
print("Controles:", ctrl)

Score fundido: 0.4200436004263153
Custo final: 0.8673943818514136
Controles: {'p_forward': 0.5799563995736847, 'cw': 439, 'rate': 0.5799563995736847}
